In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Machine learning imports
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler


### 1.1 Preprocess your Titanic training data;


In [ ]:
USE_PREPROCESSED = True  # Set to False to run preprocessing from scratch

if USE_PREPROCESSED:
    try:
        train_df = pd.read_csv('../data/preprocessed/processed_train.csv')
        test_df = pd.read_csv('../data/preprocessed/processed_test.csv')
        print("Loaded preprocessed data from CSV files")
        print(f"Training data shape: {train_df.shape}")
        print(f"Test data shape: {test_df.shape}")
        print(f"Training features: {list(train_df.columns.drop('Survived'))}")
    except FileNotFoundError:
        print("Preprocessed files not found. Switching to raw data preprocessing...")
        USE_PREPROCESSED = False

if not USE_PREPROCESSED:
    print("\nLoading raw data and starting preprocessing...")
    train_df = pd.read_csv('../data/train.csv')
    test_df = pd.read_csv('../data/test.csv')
    
    print(f"Raw training data shape: {train_df.shape}")
    print(f"Raw test data shape: {test_df.shape}")

Loaded preprocessed data from CSV files
Training data shape: (891, 9)
Test data shape: (418, 8)
Training features: ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S']


In [27]:
# Preprocessing steps (only runs if not using preprocessed data)
if not USE_PREPROCESSED:
    # Combine train and test for consistent preprocessing
    print("Step 1: Combining train and test datasets...")
    train_df['is_train'] = 1
    test_df['is_train'] = 0
    combined_df = pd.concat([train_df, test_df], ignore_index=True)
    print(f"Combined dataset shape: {combined_df.shape}")
    
    # Drop unnecessary columns
    print("\nStep 2: Dropping unnecessary columns...")
    combined_df = combined_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], errors='ignore')
    print("Dropped: PassengerId, Name, Ticket, Cabin")
    
    # Handle missing values
    print("\nStep 3: Handling missing values...")
    print(f"Columns available after dropping: {list(combined_df.columns)}")
    
    # Check and fill Age
    if 'Age' in combined_df.columns:
        combined_df['Age'] = combined_df['Age'].fillna(combined_df['Age'].median())
        print("Filled missing values in Age")
    else:
        raise KeyError("'Age' column not found in dataset. Please check your data files.")

    # Check and fill Fare
    if 'Fare' in combined_df.columns:
        combined_df['Fare'] = combined_df['Fare'].fillna(combined_df['Fare'].median())
        print("Filled missing values in Fare")
    else:
        raise KeyError("'Fare' column not found in dataset. Please check your data files.")

    # Check and fill Embarked
    if 'Embarked' in combined_df.columns:
        embarked_mode = combined_df['Embarked'].mode()
        if len(embarked_mode) > 0:
            combined_df['Embarked'] = combined_df['Embarked'].fillna(embarked_mode[0])
            print("Filled missing values in Embarked")
        else:
            combined_df['Embarked'] = combined_df['Embarked'].fillna('S')  # Default to Southampton
            print("Filled missing values in Embarked (using default 'S')")
    else:
        print("Warning: 'Embarked' column not found. This may have been preprocessed already.")
        
    # Encode categorical variables
    print("\nStep 4: Encoding categorical variables...")
    
    # Encode Sex
    if 'Sex' in combined_df.columns:
        combined_df['Sex'] = combined_df['Sex'].map({'male': 0, 'female': 1})
        print("Encoded Sex (binary)")
    else:
        raise KeyError("'Sex' column not found in dataset. Please check your data files.")
    
    # Encode Embarked if it exists
    if 'Embarked' in combined_df.columns:
        combined_df = pd.get_dummies(combined_df, columns=['Embarked'], drop_first=True, prefix='Embarked')
        print("Encoded Embarked (one-hot encoding)")
    else:
        print("Warning: 'Embarked' column not found. Skipping one-hot encoding.")
    
    # Separate back to train and test
    print("\nStep 5: Separating train and test datasets...")
    train_df = combined_df[combined_df['is_train'] == 1].drop(columns=['is_train'])
    test_df = combined_df[combined_df['is_train'] == 0].drop(columns=['is_train', 'Survived'], errors='ignore')
    
    # Convert one-hot encoded columns to int
    embarked_cols = [col for col in train_df.columns if col.startswith('Embarked_')]
    if embarked_cols:
        train_df[embarked_cols] = train_df[embarked_cols].astype(int)
        if all(col in test_df.columns for col in embarked_cols):
            test_df[embarked_cols] = test_df[embarked_cols].astype(int)
        print(f"Converted {len(embarked_cols)} Embarked columns to int")
    else:
        print("No Embarked columns found (may have been preprocessed already)")
    
    print(f"\nFinal training data shape: {train_df.shape}")
    print(f"Final test data shape: {test_df.shape}")
    print("\nPreprocessing completed!")

    # Save preprocessed data
    train_df.to_csv('../data/preprocessed/processed_train.csv', index=False)
    test_df.to_csv('../data/preprocessed/processed_test.csv', index=False)
    print("Saved preprocessed data to CSV files")
else:
    print("\nUsing preprocessed data - skipping preprocessing steps")


Using preprocessed data - skipping preprocessing steps


In [ ]:
# Display final preprocessed data summary
print("\n=== Data Summary ===")
print(f"Training set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")
print(f"\nTraining features ({len(train_df.columns)-1}):")
for col in train_df.columns:
    if col != 'Survived':
        print(f"  - {col}")
print(f"\nMissing values in training data: {train_df.isnull().sum().sum()}")
print(f"Missing values in test data: {test_df.isnull().sum().sum()}")


=== Data Summary ===
Training set shape: (891, 9)
Test set shape: (418, 8)

Training features (8):
  - Pclass
  - Sex
  - Age
  - SibSp
  - Parch
  - Fare
  - Embarked_Q
  - Embarked_S

Missing values in training data: 0
Missing values in test data: 0


In [6]:
# Prepare features and target for machine learning
X_train = train_df.drop(columns=["Survived"])
y_train = train_df["Survived"]
X_test = test_df.copy()

print("\n=== Feature Preparation ===")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"\nFeatures for modeling: {list(X_train.columns)}")
print(f"\nTarget distribution:")
print(y_train.value_counts())
print(f"\nSurvival rate: {y_train.mean():.2%}")


=== Feature Preparation ===
X_train shape: (891, 9)
y_train shape: (891,)
X_test shape: (418, 9)

Features for modeling: ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S', 'is_train']

Target distribution:
Survived
0.0    549
1.0    342
Name: count, dtype: int64

Survival rate: 38.38%


### 1.2 (5 points ) Learn and fine-tune a decision tree model with the Titanic training data, plot your decision tree;


In [14]:
print("=" * 60)
print("Task 1.2: Learning and Fine-Tuning Decision Tree Model")
print("=" * 60)

# Define the parameter grid to search (matching script.ipynb style)
dt_param_grid = {'criterion': ['gini', 'entropy'],'max_depth': [3, 5, 7, 10],'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4]}

# Use GridSearchCV for hyperparameter tuning
dt_grid = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid=dt_param_grid, cv=5, scoring='accuracy', n_jobs=-1,verbose=1)

# Fit the grid search
print("\nStarting GridSearchCV for Decision Tree...")
print("This may take a few minutes...")
dt_grid.fit(X_train, y_train)

# Get the best fine-tuned model
best_dt = dt_grid.best_estimator_

print(f"\n✓ Grid search completed!")
print(f"Best Hyperparameters: {dt_grid.best_params_}")
print(f"Best CV Score (from GridSearchCV): {dt_grid.best_score_:.4f}")

Task 1.2: Learning and Fine-Tuning Decision Tree Model

Starting GridSearchCV for Decision Tree...
This may take a few minutes...
Fitting 5 folds for each of 72 candidates, totalling 360 fits

✓ Grid search completed!
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 2}
Best CV Score (from GridSearchCV): 0.8182


In [ ]:
# Plot the best decision tree 
print("\n" + "=" * 60)
print("Plotting Decision Tree (Required for Task 1.2)")
print("=" * 60)

print("\nGenerating decision tree visualization...")
plt.figure(figsize=(30, 20), dpi=300)
plot_tree(
    best_dt, 
    feature_names=X_train.columns, 
    class_names=['Not Survived', 'Survived'], 
    filled=True, 
    rounded=True,
    fontsize=10
)

plt.title(f"Fine-Tuned Decision Tree Visualization (Max Depth = {best_dt.max_depth})", fontsize=20, pad=20)
plt.tight_layout()
plt.savefig('decision_tree.png', dpi=300, bbox_inches='tight')
print("✓ Decision tree plot saved to 'decision_tree.png'")
plt.show()


Plotting Decision Tree (Required for Task 1.2)

Generating decision tree visualization...
✓ Decision tree plot saved to 'decision_tree.png'


### 1.3 (5 points) Apply the five-fold cross validation of your fine-tuned decision tree learning model to the Titanic training data to extract average classification accuracy;


In [ ]:
print("\n" + "=" * 60)
print("Task 1.3: Decision Tree 5-Fold Cross-Validation")
print("=" * 60)

# Apply 5-fold cross-validation to the fine-tuned decision tree learning model
dt_cv_scores = cross_val_score(best_dt, X_train, y_train, cv=5, scoring='accuracy')
dt_average_accuracy = dt_cv_scores.mean()

print(f"\n5-Fold CV Scores: {dt_cv_scores}")
print(f"\n✓ Average Classification Accuracy: {dt_average_accuracy:.4f}")
print(f"  Standard Deviation: {dt_cv_scores.std():.4f}")
print(f"  95% Confidence Interval: {dt_average_accuracy:.4f} (+/- {dt_cv_scores.std() * 2:.4f})")


Task 1.3: Decision Tree 5-Fold Cross-Validation

5-Fold CV Scores: [0.79888268 0.81460674 0.84269663 0.81460674 0.82022472]

✓ Average Classification Accuracy: 0.8182
  Standard Deviation: 0.0142
  95% Confidence Interval: 0.8182 (+/- 0.0283)


### 1.4 (5 points) Apply the five-fold cross validation of your fine-tuned random forest learning model to the Titanic training data to extract average classification accuracy;


In [ ]:
print("\n" + "=" * 60)
print("Task 1.4: Learning and Fine-Tuning Random Forest Model")
print("=" * 60)

# Define the parameter grid for Random Forest (more comprehensive than script.ipynb)
rf_param_grid = {'n_estimators': [100, 200],'max_depth': [5, 7, 10, None],'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4],'criterion': ['gini', 'entropy']}

# Use GridSearchCV for hyperparameter tuning
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid=rf_param_grid, cv=5, scoring='accuracy', n_jobs=-1,verbose=1)

# Fit the grid search
print("\nStarting GridSearchCV for Random Forest...")
print("This may take a few minutes...")
rf_grid.fit(X_train, y_train)

# Get the best fine-tuned model
best_rf = rf_grid.best_estimator_

print(f"\n✓ Grid search completed!")
print(f"Best Hyperparameters: {rf_grid.best_params_}")

# Apply 5-fold cross-validation to the fine-tuned random forest learning model
print("\n" + "=" * 60)
print("Task 1.4: Random Forest 5-Fold Cross-Validation")
print("=" * 60)

print("\nApplying 5-fold cross-validation to the fine-tuned Random Forest...")
rf_cv_scores = cross_val_score(best_rf, X_train, y_train, cv=5, scoring='accuracy')
rf_average_accuracy = rf_cv_scores.mean()

print(f"\n5-Fold CV Scores: {rf_cv_scores}")
print(f"\n✓ Average Classification Accuracy: {rf_average_accuracy:.4f}")
print(f"  Standard Deviation: {rf_cv_scores.std():.4f}")
print(f"  95% Confidence Interval: {rf_average_accuracy:.4f} (+/- {rf_cv_scores.std() * 2:.4f})")


Task 1.4: Learning and Fine-Tuning Random Forest Model

Starting GridSearchCV for Random Forest...
This may take a few minutes...
Fitting 5 folds for each of 144 candidates, totalling 720 fits

✓ Grid search completed!
Best Hyperparameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}

Task 1.4: Random Forest 5-Fold Cross-Validation

Applying 5-fold cross-validation to the fine-tuned Random Forest...

5-Fold CV Scores: [0.81564246 0.81460674 0.88202247 0.80337079 0.85955056]

✓ Average Classification Accuracy: 0.8350
  Standard Deviation: 0.0303
  95% Confidence Interval: 0.8350 (+/- 0.0607)


**Task 1.5: Algorithm Comparison and Analysis**

### Accuracy Comparison (5-Fold Cross-Validation)

- **Decision Tree Average Accuracy:** 0.8182 (81.82%)
- **Random Forest Average Accuracy:** 0.8350 (83.50%)

### Determination of Better Algorithm

- **Winner:** Random Forest
- **Accuracy Difference:** 0.0168 (1.68%)

---

### **1. Model Performance**

- **Decision Tree:** 81.82% accuracy
- **Random Forest:** 83.50% accuracy
- **Difference:** 1.68 percentage points

---

### **2. Algorithm Characteristics**

- **Decision Tree:**

  - Single tree model that splits data based on feature values
  - Fine-tuned parameters: `criterion='gini'`, `max_depth=7`, `min_samples_split=2`, `min_samples_leaf=4`
  - More interpretable but prone to overfitting

- **Random Forest:**
  - Ensemble method combining 200 decision trees
  - Fine-tuned parameters: `criterion='gini'`, `max_depth=None`, `min_samples_split=5`, `min_samples_leaf=2`
  - Reduces overfitting through ensemble averaging and feature randomness

---

### **3. Analysis and Insights**

- Random Forest outperformed Decision Tree
- This aligns with general expectations because:
  - Random Forests reduce overfitting through ensemble averaging
  - Feature randomness helps improve generalization
  - Bagging approach combines multiple models for better stability

---

### **4. General Conclusions**

- Both algorithms are competitive for this classification problem
- Random Forests are typically more robust to overfitting and provide better generalization
- Decision Trees offer better interpretability but require careful tuning
- The small performance difference suggests the dataset is well-suited for tree-based methods
- Further hyperparameter tuning or feature engineering could improve both models

---

### **5. Final Recommendation**

- **Random Forest** is the better choice for this dataset
- It provides better generalization and reduced overfitting risk
- The ensemble approach is more reliable for unseen data
